In [ ]:
import getpass
import os

try:
    # load environment variables from .env file (requires `python-dotenv`)
    from dotenv import load_dotenv

    load_dotenv()
except ImportError:
    pass

os.environ["LANGSMITH_TRACING"] = "true"
if "LANGSMITH_API_KEY" not in os.environ:
    os.environ["LANGSMITH_API_KEY"] = getpass.getpass(
        prompt="Enter your LangSmith API key (optional): "
    )
if "LANGSMITH_PROJECT" not in os.environ:
    os.environ["LANGSMITH_PROJECT"] = getpass.getpass(
        prompt='Enter your LangSmith Project Name (default = "default"): '
    )
    if not os.environ.get("LANGSMITH_PROJECT"):
        os.environ["LANGSMITH_PROJECT"] = "default"
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass(
        prompt="Enter your OpenAI API key (required if using OpenAI): "
    )
print(os.environ["OPENAI_API_KEY"])

base_url = os.environ.get("BASE_URL")
model_name = os.environ.get("MODEL_NAME")
deepseek_model_name = os.environ.get("DEEPSEEK_MODEL_NAME")

print(base_url, model_name)
print("langsmith api key", os.environ["LANGSMITH_API_KEY"])

In [4]:
if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("enter api key for openai: ")

from langchain.chat_models import init_chat_model
from langchain_deepseek import ChatDeepSeek


llm = init_chat_model(model_name, model_provider="openai", base_url=base_url)

llm_deepseek = ChatDeepSeek(
    model=deepseek_model_name,
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    base_url=base_url,
)


In [5]:
# test
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage("Translate the following from English into Chinese"),
    HumanMessage("hi!"),
]

llm.invoke(messages)

AIMessage(content='你好！', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 22, 'total_tokens': 24, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'qwen-plus', 'system_fingerprint': None, 'id': 'chatcmpl-221e7ba8-fdd1-9a95-b23f-ae184fa18648', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--92e23a43-1765-48c9-b1a8-8aab205b0874-0', usage_metadata={'input_tokens': 22, 'output_tokens': 2, 'total_tokens': 24, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})

In [6]:
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field

tagging_prompt = ChatPromptTemplate.from_template(
    """
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'Classification' function.

Passage:
{input}
"""
)


class Classification(BaseModel):
    sentiment: str = Field(description="The sentiment of the text")
    aggressiveness: int = Field(
        description="How aggressive the text is on a scale from 1 to 10"
    )
    language: str = Field(description="The language the text is written in")


# Structured LLM
structured_llm = llm_deepseek.with_structured_output(Classification)

In [7]:
inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
prompt = tagging_prompt.invoke({"input": inp})
print(prompt)
response = llm.invoke(prompt)

messages=[HumanMessage(content="\nExtract the desired information from the following passage.\n\nOnly extract the properties mentioned in the 'Classification' function.\n\nPassage:\nEstoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!\n", additional_kwargs={}, response_metadata={})]


In [23]:
print(response)

content='```json\n{\n    "Classification": {\n        "Emotion": "Happy",\n        "Relationship": "Friendship"\n    }\n}\n``` \n\nNote: The properties "Emotion" and "Relationship" are assumed based on typical classification tasks for such a passage. If specific properties were intended, they would need to be clarified.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 67, 'prompt_tokens': 62, 'total_tokens': 129, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'qwen-plus', 'system_fingerprint': None, 'id': 'chatcmpl-63f0b8e1-ff2d-9712-962d-252da0dfd486', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None} id='run--1933dd87-5a3d-4fe3-8ec1-e55abca39456-0' usage_metadata={'input_tokens': 62, 'output_tokens': 67, 'total_tokens': 129, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}}


In [8]:
# Structured LLM
# structured_llm = llm_deepseek.with_structured_output(Classification)
response = structured_llm.invoke(prompt)

In [9]:
response

Classification(sentiment='positive', aggressiveness=1, language='Spanish')

In [36]:
llm_deepseek.invoke(prompt).model_dump_json()

'{"content":"```json\\n{\\n  \\"Classification\\": {}\\n}\\n```","additional_kwargs":{"refusal":null},"response_metadata":{"token_usage":{"completion_tokens":11,"prompt_tokens":57,"total_tokens":68,"completion_tokens_details":null,"prompt_tokens_details":{"audio_tokens":null,"cached_tokens":0},"prompt_cache_hit_tokens":0,"prompt_cache_miss_tokens":57},"model_name":"deepseek-chat","system_fingerprint":"fp_8802369eaa_prod0425fp8","id":"51feb995-d074-4041-9ab1-4cbc79e218f4","service_tier":null,"finish_reason":"stop","logprobs":null},"type":"ai","name":null,"id":"run--181f8ed3-13ff-4b97-914a-98edaa799bef-0","example":false,"tool_calls":[],"invalid_tool_calls":[],"usage_metadata":{"input_tokens":57,"output_tokens":11,"total_tokens":68,"input_token_details":{"cache_read":0},"output_token_details":{}}}'

## 百炼大模型集成

In [39]:
from langchain_openai import ChatOpenAI
import os

chatLLM = ChatOpenAI(
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    model="qwen-plus",  # 此处以qwen-plus为例，您可按需更换模型名称。模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
    # response_format={"type": "json_object"},
    # other params...
)
chatLLM.invoke(prompt).model_dump_json()

/Users/zhangjintao/my-work/llm-demo/.venv/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3610: UserWarning: WARNING! response_format is not default parameter.
                response_format was transferred to model_kwargs.
                Please confirm that response_format is what you intended.
  if await self.run_code(code, result, async_=asy):


BadRequestError: Error code: 400 - {'error': {'code': 'invalid_parameter_error', 'param': None, 'message': "<400> InternalError.Algo.InvalidParameter: 'messages' must contain the word 'json' in some form, to use 'response_format' of type 'json_object'.", 'type': 'invalid_request_error'}, 'id': 'chatcmpl-b6f28179-19fd-9bd3-b353-76771c4691d0', 'request_id': 'b6f28179-19fd-9bd3-b353-76771c4691d0'}

In [38]:
llm_s = chatLLM.with_structured_output(None, method="json_mode")
response_s = llm_s.invoke(prompt)

BadRequestError: Error code: 400 - {'error': {'code': 'invalid_parameter_error', 'param': None, 'message': "<400> InternalError.Algo.InvalidParameter: 'messages' must contain the word 'json' in some form, to use 'response_format' of type 'json_object'.", 'type': 'invalid_request_error'}, 'id': 'chatcmpl-7fc4cfe7-9353-97dd-a38b-7ae7c6cedada', 'request_id': '7fc4cfe7-9353-97dd-a38b-7ae7c6cedada'}

In [33]:
response_s

{' Classification ': 'positive'}